                                                                                                                        © Moon

# Project Report: Evaluation of the Centrality Algorithm, PageRank

## Introduction

Most important vertices can be 
PageRank algorithm is an algorithm to identify most important vertices in a graph. Examples of application of the PR algorithm are analytics, web, social networks, and etc.